CASOS RECUPERADOS e ACOMPANHAMENTO

In [1]:
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession

In [10]:
dados_sem_schema = spark.read.csv("/user/milena/covid/dados.csv")

In [11]:
print(dados_sem_schema.printSchema())
dados_sem_schema.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)

None
+------+------+----------+---------+--------+------+-------+-------+-----------+--------------+---------+
|   _c0|   _c1|       _c2|      _c3|     _c4|   _c5|    _c6|    _c7|        _c8|           _c9|     _c10|
+------+------+----------+---------+--------+------+-------+-------+-----------+--------------+---------+
|regiao|estado|      data|      pop|  casosA|casosN|obitosA|obitosN|recuperados|acompanhamento|municipio|
|Brasil|  null|2022-01-01|210147125|22291507|  3986| 619105|     49|   21581668|         90734|     null|
|Brasil|  null|2022-01-02|210147125|22293228|  1721| 619133|     28|   21

In [3]:
estrutura_dados = [
    
    StructField("regiao", IntegerType()),
    StructField("estado", IntegerType()),
    StructField("data", IntegerType()),
    StructField("pop", IntegerType()),
    StructField("casosA", IntegerType()),
    StructField("casosN", IntegerType()),
    StructField("obitosA", IntegerType()),
    StructField("obitosN", IntegerType()),
    StructField("recuperados", IntegerType()),
    StructField("acompanhamento", IntegerType()) 
    
]

schema_names = StructType(estrutura_dados)

In [8]:
tabela1 = spark.read.csv("/user/milena/covid/dados.csv",schema=schema_names)

In [9]:
tabela1 = tabela1.agg({'acompanhamento': 'sum', 'recuperados': 'sum'}).show()

+-------------------+----------------+
|sum(acompanhamento)|sum(recuperados)|
+-------------------+----------------+
|          172064706|      4925750922|
+-------------------+----------------+

